# Introduction: sandbox for developing digital water systems

## Objective

The goal of this notebook is to familiazie one self with the simulation environment and tools used in the workshop for building digital twins.

### Tools of the trade
**Explain what each tool does**
1. pyswmm
2. swmmio
3. pystroms
4. matplotlib
5. numpy
6. pandas
7. scikit-learn
8. pytorch

In [3]:
import pyswmm
import swmmio
import pystorms
import matplotlib.pyplot as plt

TODO: Add a image of the network or networks we will be using in this example.


Link to the where they can find more information about the networks.


Reach out to the ASCE to get a list of the names of the people so we can communicate information with them. 

### Simulating a stormwater network in EPA-SWMM via python

In [9]:
# Load a input file
input_file = pystorms.load_network("theta")

In [10]:
# pystorms provides a collection of real-life inspired stormwater networks.
# In this example, we are just loading a network.
print(f"SWMM input file path: {input_file}")

SWMM input file path: /Users/pluto/Archive/dev/ewri2023_DigitalWater101/venv/lib/python3.9/site-packages/pystorms/networks/theta.inp


👨🏼‍🎓 Bonus Question: Try running a simulation with an other stormwater network from pystorms or may be one of your own stormwater networks.

In [11]:
# Once we have the stormwater network, we hand over the input file to the awesome pyswmm
# pyswmm helps us control EPA-SWMM from python, so that we don't have to press buttons anymore 😆
simulation = pyswmm.Simulation(inputfile=input_file)

# run the simulation
simulation.execute()


... SWMM Version 5.2.3
 o  Retrieving project data
 o  Simulating day: 0     hour:  00     hour: 1 0     hour: 2 0     hour: 3 0     hour: 4 0     hour: 5 0     hour: 6 0     hour: 7 0     hour: 8 0     hour: 9 0     hour: 100     hour: 110     hour: 120     hour: 130     hour: 140     hour: 150     hour: 160     hour: 170     hour: 180     hour: 190     hour: 200     hour: 210     hour: 220     hour: 231     hour: 0 1     hour: 1 1     hour: 2 1     hour: 3 1     hour: 4 1     hour: 5 1     hour: 6 1     hour: 7 1     hour: 8 1   

Okay, this is cool, but why do we need python to do this? I can run this by using good old EPA-SWMM GUI

In [16]:
# pyswmm gives us the power to intervean during the simulation and make changes

simulation = pyswmm.Simulation(inputfile=input_file)

for step in simulation:
    pass

simulation.close()